In [65]:
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import os

from photutils.aperture import aperture_photometry, CircularAperture
from vip_hci.preproc.derotation import cube_derotate
from vip_hci.metrics import contrast_curve
from vip_hci.var import fit_2dgaussian
from vip_hci.psfsub import pca, pca_annular
from astropy.io import fits

from core.engine import get_angle_radius, preprocess

In [57]:
ccurve = pd.read_csv('./logs/f_dhtau/contrast_curve.txt', names=['1', '2'])

In [58]:
data_path = './data/real/f_dhtau/'
table, cube, npsf, rot_angles, backmoments = preprocess(data_path)
single_npsf = npsf[0]
rot_angles = -rot_angles
cube = cube_derotate(cube, rot_angles, nproc=4, imlib='opencv', interpolation='nearneig')

exp_path = './logs/f_dhtau/'
comp_parameters = pd.read_csv(os.path.join(exp_path, 'prediction.csv'))
row = comp_parameters.iloc[0]

We first need to estimate the flux of the star. We load the PSF

In [59]:
psf_route  = os.path.join(data_path, 'median_unsat.fits')
psf_list   = fits.getdata(psf_route, ext=0)
single_psf = psf_list[0, 0]

now we fit a 2d-gaussian model to the PSF  

In [60]:
fit = fit_2dgaussian(single_psf, 
                     crop=True, 
                     cropsize=30, 
                     debug=False, 
                     full_output=True)  
mean_fwhm_fit = np.mean([fit.fwhm_y, fit.fwhm_x])

In [62]:
aper = CircularAperture((single_npsf[0].shape[0]//2, single_npsf[0].shape[0]//2), 
                        r=mean_fwhm_fit/2.) 
starphot_estimated = aperture_photometry(single_npsf, aper, method='exact')
starphot_estimated = starphot_estimated['aperture_sum'][0]

In [ ]:
cc_1 = contrast_curve(cube, 
                      rot_angles, 
                      psf_norm, 
                      fwhm=row['fwhm_mean'], 
                      pxscale=0.01225, 
                      starphot=starphot_estimated,
                      sigma=5, 
                      nbranch=1, 
                      algo=pca_annular,
                      radius_int=int(row['fwhm_mean']),
                      ncomp=9, 
                      debug=True)

――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
Starting time: 2023-08-01 16:27:26
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
ALGO : pca_annular, FWHM = 4.261339461993815, # BRANCHES = 1, SIGMA = 5, STARPHOT = 0.9999999207732931
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
